In [2]:
import re
import urllib.parse
import requests
import pandas as pd
import pyodbc

In [3]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"

def parse_search_page(address:str)->str:
    """Производит поиск информации по запросу интересующего адреса в поисковой системе google
    Входной параметр: адрес объекта, для которого нужно произвести поиск информации
    Возвращаемый результат: шаблон первой страницы поискового запроса в системе google в формате html
    """

    URL = "https://google.com/search?q={0}".format(urllib.parse.quote_plus("2gis "+address))
    resp = requests.get(URL, headers={"user-agent": USER_AGENT})
    response = str(resp.text) if resp.status_code==200 else ""
    with open('ress.html','w',encoding='utf-8') as file:
        file.write(resp.text)
    return response

def search_link(search_page_template:str)->str:
    """Производит поиск ссылки на сайт 2gis
    Входной параметр: шаблон первой страницы поискового запроса
    Возвращаемый результат: ссылка на необходимый объект на сайте 2gis
    """

    link = re.findall(r'href=\"(http[s]?:\/\/2gis.ru\/\S+\/geo[^\"]*)\"', search_page_template)
    res_link = link[0] if len(link)>0 else ""
    return res_link

def search_info(organizations_list):
    """Производит обработку словарей с информацией об организациях, хранящихся в строковом представлении
    Входной параметр: список строк с информацией об организации в формате словаря
    Возвращаемый результат: строка со всеми предприятиями
    """
    organizations = []
    for organization in organizations_list:
        dictionary = eval(organization)
        organization_type = dictionary.get(
            'extension',
            dictionary.get('description','Не указан тип организации')
        ).capitalize()
        organization_name = dictionary.get('primary')
        organizations.append(organization_type+"-"+organization_name)
    return ','.join(organizations)

def parse_2gis_page(link:str)->str:
    """Производит поиск информации об организациях, размещенных в представленном объекте
    Входной параметр: ссылка на страницу объекта на сайте 2gis
    Возвращаемый результат: все предприятия (организации),адрес для которого производился поиск 
    """
    
    organizations = "В данном месте нет организаций"
    address_name = []
    res_address = "Проблемы с получением адреса"    
    if len(link)>0:
        response = requests.get(link+'/tab/inside', headers={"user-agent": USER_AGENT})
        if response.status_code ==200:
            organizations_list = re.findall(r'\"name_ex\"\s?:\s?(\{[^\}]*\}),',response.text)
            address_name = re.findall(r'\"full_name\"\s?:\s?([^\}]*),',response.text)
            if len(address_name)>0:
                res_address = address_name[0].split("\"")[1]
            else:
                address_name = re.findall(r'\"address_name\"\s?:\s?\"(.*)\"',response.text)
                if len(address_name)>0:
                    res_address = address_name[0].split("\"")[1]
            if len(organizations_list)>0:
                organizations = search_info(organizations_list)
    return organizations, res_address

In [5]:
class DbWorker:

    def __init__(self, connection_string):
        self.connection_string = connection_string


    def search_addresses(self, file_path, f_column, s_column):
        """Производит поиск уникальных адресов в столбцах excel-файла
        input: путь к файлу, название первого столбца, название второго столбца
        output: список уникальных значений адресов
        """

        df = pd.read_excel(file_path)
        addresses = df[f_column].tolist()+df[s_column].tolist()
        return list(set(addresses))

    def insert_addresses_data(self, address):
        """Вносит запись об адресе в таблицу all_addresses
        input: адрес местоположения
        """
        connection = pyodbc.connect(self.connection_string, autocommit=True)
        request = """INSERT INTO all_addresses (address,status) VALUES (?,0)"""
        cursor = connection.cursor()
        cursor.execute(request, str(address))
        connection.commit()
        connection.close()
            
    def insert_addresses_info(self, address, organizations, searched_address, link):
        """Вносит запись об адресе в таблицу addresses_info
        input: адрес местоположения
        """
        connection = pyodbc.connect(self.connection_string, autocommit=True)
        request = """INSERT INTO addresses_info (address, organizations, searched_address, link) VALUES (?,?,?,?)"""
        cursor = connection.cursor()
        cursor.execute(request, str(address), str(organizations), str(searched_address), str(link))
        connection.commit()
        connection.close()

    def update_addresses_table(self, address):
        """Обновляет данные в таблице с адресами
        input: адрес, для которого необходимо произвести обновление данных
        """
        connection = pyodbc.connect(self.connection_string, autocommit=True)
        request = """update all_addresses set status = 1 where address = ?"""
        cursor = connection.cursor()
        cursor.execute(request, str(address))
        connection.commit()
        connection.close()
    
    def search_addresses_data(self):
        connection = pyodbc.connect(self.connection_string, autocommit=True)
        request = """select address from all_addresses where status = 0"""
        cursor = connection.cursor()
        cursor.execute(request)
        for row in cursor:
            fp = parse_search_page(row.address) #fp - first page
            link = search_link(fp)
            organizations, searched_address = parse_2gis_page(link)
            self.insert_addresses_info(row.address, organizations,searched_address,link)
            self.update_addresses_table(row.address)
        connection.commit()
        connection.close()


In [6]:
con_str = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=addresses;Trusted_Connection=yes;"
db = DbWorker(con_str)

In [7]:
addresses_list = db.search_addresses('Addresses.xlsx','Адрес пункта отправления','Адрес пункта назначения')

KeyboardInterrupt: 

In [17]:
for address in addresses_list:
    db.insert_addresses_data(address)

In [8]:
db.search_addresses_data()